In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
from scipy.interpolate import make_interp_spline


## Desktop CPU plots

### Chip Metrics Plot
*Tot CFP, Mfg CFP %, Ope CFP %, CFPA, Process Node, DieSize*

In [2]:
def set_plot_ax(ax):
    ax.yaxis.label.set_size(14)
    ax.yaxis.label.set_fontweight('bold')
    ax.xaxis.label.set_size(14)
    ax.xaxis.label.set_fontweight('bold')
    ax.title.set_fontsize(16)
    ax.title.set_fontweight('bold')
    for label in ax.get_yticklabels():
        label.set_fontsize(12)
        label.set_fontweight('bold')

    for tick in ax.get_xticklabels():
        # tick.set_rotation(15)
        tick.set_fontsize(12)
        tick.set_fontweight('bold')

    for label in ax.get_yticklabels():
        label.set_fontweight('bold')
        label.set_fontsize(12)  


In [ ]:
def fake_arr(arr):

    fake = np.zeros(len(arr) * 2 - 1)
    for i in range(len(arr)):
        fake[i * 2] = arr[i]
        if i < len(arr) - 1:
            fake[i * 2 + 1] = (arr[i] + arr[i + 1]) / 2

    return fake

fake_arr([1, 2, 3, 4, 5])

In [ ]:
def fake_arr2(arr):
    # First iteration: Double the points by adding midpoints
    fake = np.zeros(len(arr) * 2 - 1)
    for i in range(len(arr)):
        fake[i * 2] = arr[i]
        if i < len(arr) - 1:
            fake[i * 2 + 1] = (arr[i] + arr[i + 1]) / 2
    
    # Second iteration: Double the points again by adding midpoints
    result = np.zeros(len(fake) * 2 - 1)
    for i in range(len(fake)):
        result[i * 2] = fake[i]
        if i < len(fake) - 1:
            result[i * 2 + 1] = (fake[i] + fake[i + 1]) / 2
    
    return result


fake_arr2([1, 2, 3, 4, 5])

In [5]:
def chip_metrics_plot(nv, server=True):
    # Define style constants for easy tuning
    FACTOR = 2
    # TITLE_FONT_SIZE = 22 * FACTOR
    # LABEL_FONT_SIZE = 18 * FACTOR
    TICK_FONT_SIZE = 14 * FACTOR
    AXIS_LABEL_FONT_SIZE = 16 * FACTOR
    LEGEND_FONT_SIZE = 15 * FACTOR
    LINE_WIDTH = 5 * FACTOR
    MARKER_SIZE = 10 * FACTOR
    GRID_ALPHA = 0.2
    LABEL_COLOR = '#4a4a4a'
    SPLINE = 3  # Spline interpolation factor


    # Colors from the provided palette

    COLOR_5 = '#248F24'  # Light green
    COLOR_2 = '#D62728'  # Orange-yellow
    COLOR_6 = '#457B9D'  # Muted sky blue


    plot_title = 'Datacenter CPUs Sustainability Metrics' if server else 'Desktop CPUs Sustainability Metrics'
    x_label = 'Intel Datacenter CPUs' if server else 'Intel Desktop CPUs'

    # Create figure and main axis
    fig1, ax1 = plt.subplots(figsize=(20, 12))
    # ax1.set_title(plot_title, fontsize=TITLE_FONT_SIZE, fontweight='bold', color=LABEL_COLOR, pad=20)
    # ax1.set_xlabel(x_label, fontsize=LABEL_FONT_SIZE, fontweight='bold', color=LABEL_COLOR, labelpad=15)


    # Prepare data
    x_data_fake = fake_arr(np.arange(len(nv['Product'])))  # Using indices for smooth interpolation
    x_data_fake2 = fake_arr2(np.arange(len(nv['Product'])))
    x_data = np.arange(len(nv['Product']))  # Original x-axis
    x_labels = nv['Product']  # Original product labels
    x_smooth = np.linspace(x_data_fake.min(), x_data_fake.max(), 500)  # Smoother x-axis

    # Manufacture CFP percentage (smoothened line with original markers)
    y_mfg_cfp_fake = fake_arr(nv['Emb CFP percentage'])
    y_mfg_cfp = (nv['Emb CFP percentage'])
    spline_mfg = make_interp_spline(x_data_fake, y_mfg_cfp_fake, k=SPLINE)
    y_mfg_smooth = spline_mfg(x_smooth)
    ax1.set_ylabel(ylabel='CFP Percentage', fontsize=AXIS_LABEL_FONT_SIZE, fontweight='bold', color=COLOR_2)  
    ax1.plot(x_smooth, y_mfg_smooth, label='%ECFP', color=COLOR_2, linewidth=LINE_WIDTH)
    ax1.plot(x_data, y_mfg_cfp, 's', markersize=MARKER_SIZE, color=COLOR_2, alpha=0.8)  # Plot markers at original points

    # Operational CFP percentage (smoothened line with original markers)
    y_ope_cfp = nv['Ope CFP percentage']
    spline_ope = make_interp_spline(x_data, y_ope_cfp, k=SPLINE)
    y_ope_smooth = spline_ope(x_smooth)
    ax1.plot(x_smooth, y_ope_smooth, label='%OCFP', color=COLOR_2, linestyle='--', linewidth=LINE_WIDTH)  # Orange-yellow
    ax1.plot(x_data, y_ope_cfp, 's', markersize=MARKER_SIZE, color=COLOR_2, alpha=0.8)  # Plot markers at original points

    ax1.set_xticks(x_data)  # Set x-ticks to original positions
    ax1.set_xticklabels(x_labels, fontsize=TICK_FONT_SIZE, fontweight='bold', color=LABEL_COLOR)  # Set original product names as labels

    ax1.tick_params(axis='x', rotation=15)
    ax1.tick_params(axis='both', which='major', labelsize=TICK_FONT_SIZE, colors=LABEL_COLOR)

    # Add a light grid for better readability
    ax1.grid(visible=True, linestyle='--', linewidth=1, alpha=GRID_ALPHA)

    ax1.spines['left'].set_linewidth(2)
    ax1.spines['left'].set_color(COLOR_2)
    ax1.spines['left'].set_position(('outward', 20))

    ax1.spines['bottom'].set_linewidth(2)
    ax1.spines['right'].set_visible(False)
    ax1.spines['top'].set_visible(False)

    # Set tick labels to be bold for the left y-axis (CFP Percentage %)
    for label in ax1.get_yticklabels():
        label.set_color(COLOR_2)
        label.set_fontweight('bold')
        label.set_fontsize(TICK_FONT_SIZE)

    # Release Date (Secondary X Axis)
    ax2 = ax1.twiny()
    ax2.set_xlim(ax1.get_xlim())
    ax2.set_xticks(x_data)
    ax2.set_xlabel('Release Year', fontsize=AXIS_LABEL_FONT_SIZE, fontweight='bold', color=LABEL_COLOR, labelpad=10)
    ax2.set_xticklabels(nv['Release Date'], fontsize=TICK_FONT_SIZE, fontweight='bold', color=LABEL_COLOR)
    ax2.spines['top'].set_linewidth(2)
    ax2.spines['right'].set_visible(False)
    ax2.spines['left'].set_visible(False)

    # Process Node (Tertiary X Axis)
    ax3 = ax1.twiny()
    ax3.set_xlim(ax1.get_xlim())
    ax3.set_xticks(x_data)
    ax3.set_xlabel('Process Node(nm)', fontsize=AXIS_LABEL_FONT_SIZE, fontweight='bold', color=LABEL_COLOR, labelpad=10)
    ax3.set_xticklabels(nv['Process Size (nm)'], fontsize=TICK_FONT_SIZE, fontweight='bold', color=LABEL_COLOR)
    ax3.spines['top'].set_linewidth(2)
    ax3.spines['top'].set_position(('outward', 80))
    ax3.spines['right'].set_visible(False)
    ax3.spines['left'].set_visible(False)

# Re-apply settings to ensure font size is correctly applied
    for label in ax3.get_xticklabels():
        label.set_fontsize(TICK_FONT_SIZE)
        label.set_fontweight('bold')
        label.set_color(LABEL_COLOR)

    # Total CFP (smoothened line with original markers)
    cfp_color = COLOR_6  # Blue
    y_tot_cfp_fake = fake_arr(nv['Tot CFP'])
    y_tot_cfp = nv['Tot CFP']
    spline_tot_cfp = make_interp_spline(x_data_fake, y_tot_cfp_fake, k=SPLINE)
    y_tot_smooth = spline_tot_cfp(x_smooth)
    ax4 = ax1.twinx()
    ax4.set_ylabel(r'CFP $(kg\ CO_2 e.q.)$', fontsize=AXIS_LABEL_FONT_SIZE, fontweight='bold', color=cfp_color, labelpad=15)
    ax4.plot(x_smooth, y_tot_smooth, label='Total CFP', color=cfp_color, linewidth=LINE_WIDTH)
    ax4.plot(x_data, y_tot_cfp, 's', markersize=MARKER_SIZE, color=cfp_color, alpha=0.8)  # Plot markers at original points
    ax4.spines['right'].set_color(cfp_color)
    ax4.spines['right'].set_linewidth(2)
    ax4.spines['right'].set_position(('outward', 30))
    ax4.spines['left'].set_visible(False)
    ax4.spines['top'].set_visible(False)

    for label in ax4.get_yticklabels():
        label.set_color(cfp_color)
        label.set_fontweight('bold')
        label.set_fontsize(TICK_FONT_SIZE)

    # Die Size (smoothened line with original markers)
    die_color = COLOR_5  # Medium green
    y_die_size_fake = fake_arr(nv['DieSizeValue'])
    y_die_size = nv['DieSizeValue']
    spline_die_size = make_interp_spline(x_data_fake, y_die_size_fake, k=SPLINE)
    y_die_smooth = spline_die_size(x_smooth)
    ax6 = ax1.twinx()
    ax6.set_ylabel(r'Die Area($mm^2$)', fontsize=AXIS_LABEL_FONT_SIZE, fontweight='bold', color=die_color, labelpad=15)
    ax6.plot(x_smooth, y_die_smooth, label='Die Area', color=die_color, linewidth=LINE_WIDTH)
    ax6.plot(x_data, y_die_size, 's', markersize=MARKER_SIZE, color=die_color, alpha=0.8)  # Plot markers at original points
    ax6.yaxis.set_label_position('left')
    ax6.yaxis.set_ticks_position('left')
    ax6.spines['left'].set_color(die_color)
    ax6.spines['left'].set_linewidth(2)
    ax6.spines['left'].set_position(('outward', 140))
    ax6.spines['right'].set_visible(False)
    ax6.spines['top'].set_visible(False)

    for label in ax6.get_yticklabels():
        label.set_color(die_color)
        label.set_fontweight('bold')
        label.set_fontsize(TICK_FONT_SIZE)

    # Add legend

    if server:
        fig1.legend(loc='upper center', fontsize=AXIS_LABEL_FONT_SIZE, frameon=False, ncol=1, bbox_to_anchor=(0.3, 0.8))

    else:
        fig1.legend(loc='upper center', fontsize=AXIS_LABEL_FONT_SIZE, frameon=False, ncol=1, bbox_to_anchor=(0.8, 0.8))

    return fig1


In [6]:
def performance_metric_plot(nv, server=True):
    # Define style constants for easy tuning
    FACTOR = 2
    # TITLE_FONT_SIZE = 22 * FACTOR
    # LABEL_FONT_SIZE = 18 * FACTOR
    TICK_FONT_SIZE = 14 * FACTOR
    AXIS_LABEL_FONT_SIZE = 16 * FACTOR
    LEGEND_FONT_SIZE = 15 * FACTOR
    LINE_WIDTH = 5 * FACTOR
    MARKER_SIZE = 10 * FACTOR
    GRID_ALPHA = 0.2
    LABEL_COLOR = '#4a4a4a'

    SPLINE = 3  # Spline interpolation factor
    # Colors from the provided palette

    # COLOR_1 = '#B02425'  # Dark red
    COLOR_1 = '#FF5252'  # 

    COLOR_2 = '#FFA500'  # Orange-yellow (corrected to actual orange-yellow)
    COLOR_4 = '#248F24'  # Light green (added as missing COLOR_3)
    COLOR_5 = '#4169E1'  # Medium blue (corrected description from "medium green")
    COLOR_3 = '#4F4F4F'  # Dark color

    # COLOR1 = COLOR_4
    # COLOR_2 = COLOR_3
    # COLOR_4 = COLOR_3
    # COLOR_5 = COLOR_3

    SMOOTH_FACTOR = 3

    # Plot and axis labels
    plot_title = 'Datacenter CPUs Normalized Performance Efficiency' if server else 'Desktop CPUs Normalized Performance Efficiency'
    x_label = 'Intel Datacenter CPUs' if server else 'Intel Desktop CPUs'

    # Auto-set figure size based on input data
    fig_width = 20  # Set a minimum width of 10, scale with number of products
    fig_height = 12  # Fixed height for better readability

    # Create figure and main axis
    fig1, ax1 = plt.subplots(figsize=(fig_width, fig_height))
    # ax1.set_title(plot_title, fontsize=TITLE_FONT_SIZE, fontweight='bold', color=LABEL_COLOR, pad=20)
    # ax1.set_xlabel(x_label, fontsize=LABEL_FONT_SIZE, fontweight='bold', color=LABEL_COLOR, labelpad=15)

    # Adjust spacing to avoid overlap
    plt.subplots_adjust(top=0.85, bottom=0.1, left=0.1, right=0.9, hspace=0.3)

    # Prepare data
    x_data_fake = fake_arr(np.arange(len(nv['Product'])))  # Using indices for smooth interpolation
    x_data_fake2 = fake_arr2(np.arange(len(nv['Product'])))
    x_data = np.arange(len(nv['Product']))  # Using indices for smooth interpolation
    x_labels = nv['Product']  # Original product labels
    x_smooth = np.linspace(x_data.min(), x_data.max(), 500)  # Smoother x-axis

    # OpenCL Score (smoothened line with original markers)
    y_opencl_score = nv['PassMark CPU score'] if server else nv['Multi-Core Score']
    spline_opencl = make_interp_spline(x_data, y_opencl_score, k=SMOOTH_FACTOR)
    y_opencl_smooth = spline_opencl(x_smooth)
    score_color = COLOR_5
    ax1.set_ylabel('Normalized Factor', fontsize=AXIS_LABEL_FONT_SIZE, fontweight='bold', color=LABEL_COLOR)
    # ax1.set_ylabel('Performance', fontsize=AXIS_LABEL_FONT_SIZE, fontweight='bold')
    ax1.plot(x_smooth, y_opencl_smooth, label='Performance', color=score_color, linewidth=LINE_WIDTH)
    ax1.plot(x_data, y_opencl_score, 's', markersize=MARKER_SIZE, color=score_color, alpha=0.8)

    # Performance / Total CFP on the same y-axis
    cfp_color = COLOR_4
    y_opencl_cfp = nv['Score / CFP'] if server else nv['Multi-Core Score / CFP']
    spline_opencl_cfp = make_interp_spline(x_data, y_opencl_cfp, k=SMOOTH_FACTOR)
    y_opencl_cfp_smooth = spline_opencl_cfp(x_smooth)
    ax1.plot(x_smooth, y_opencl_cfp_smooth, label='Performance / CFP', color=cfp_color, linewidth=LINE_WIDTH)
    ax1.plot(x_data, y_opencl_cfp, 's', markersize=MARKER_SIZE, color=cfp_color, alpha=0.8)

    # Performance / Embodied CFPA on the same y-axis
    transistor_color = COLOR_2
    y_trans_score = nv['Score / ECFPA'] if server else nv['Multi-Core Score / ECFPA']
    spline_trans_score = make_interp_spline(x_data, y_trans_score, k=SMOOTH_FACTOR)
    y_trans_score_smooth = spline_trans_score(x_smooth)
    ax1.plot(x_smooth, y_trans_score_smooth, label='Performance / Embodied CFPA', linestyle = '--', color=transistor_color, linewidth=LINE_WIDTH)
    ax1.plot(x_data, y_trans_score, 's', markersize=MARKER_SIZE, color=transistor_color, alpha=0.8)

    # Axis configuration
    ax1.set_xticks(x_data)  # Set x-ticks to original positions
    ax1.set_xticklabels(x_labels, fontsize=TICK_FONT_SIZE, fontweight='bold', color=LABEL_COLOR)
    ax1.tick_params(axis='both', which='major', labelsize=TICK_FONT_SIZE, colors=LABEL_COLOR)
    ax1.tick_params(axis='x', rotation=15)

    # Add a light grid for better readability
    ax1.grid(visible=True, linestyle='--', linewidth=1, alpha=GRID_ALPHA)

    # Spine settings
    # ax1.spines['left'].set_color(score_color)
    ax1.spines['left'].set_linewidth(2)
    ax1.spines['left'].set_position(('outward', 20))
    ax1.spines['bottom'].set_linewidth(2)
    ax1.spines['right'].set_visible(False)
    ax1.spines['top'].set_visible(False)

    for label in ax1.get_yticklabels():
        # label.set_color(score_color)
        label.set_fontweight('bold')
        label.set_fontsize(TICK_FONT_SIZE)

    # Release Date (Secondary X Axis)
    ax2 = ax1.twiny()
    ax2.set_xlim(ax1.get_xlim()[0] - 0.1, ax1.get_xlim()[1])  # Extend the range at the start for separation
    ax2.set_xticks(x_data)
    ax2.set_xlabel('Release Year', fontsize=AXIS_LABEL_FONT_SIZE, fontweight='bold', color=LABEL_COLOR, labelpad=10)
    ax2.set_xticklabels(nv['Release Date'], fontsize=TICK_FONT_SIZE, fontweight='bold', color=LABEL_COLOR)
    ax2.spines['top'].set_linewidth(2)
    ax2.spines['right'].set_visible(False)
    ax2.spines['left'].set_visible(False)

    # Process Node (Tertiary X Axis)
    
    ax3 = ax1.twiny()
    ax3.set_xlim(ax1.get_xlim())
    ax3.set_xticks(x_data)
    ax3.set_xlabel('Process Node(nm)', fontsize=AXIS_LABEL_FONT_SIZE, fontweight='bold', color=LABEL_COLOR, labelpad=10)
    ax3.set_xticklabels(nv['Process Size (nm)'], fontsize=TICK_FONT_SIZE, fontweight='bold', color=LABEL_COLOR)
    ax3.spines['top'].set_linewidth(2)
    ax3.spines['top'].set_position(('outward', 80))
    ax3.spines['right'].set_visible(False)
    ax3.spines['left'].set_visible(False)

    for label in ax3.get_xticklabels():
        label.set_fontsize(TICK_FONT_SIZE)
        label.set_fontweight('bold')
        label.set_color(LABEL_COLOR)

    # Embodied CFP per unit area (separate y-axis)
    y_cpfa_fake = fake_arr2(nv['ECFPA'])
    y_cpfa = nv['ECFPA']
    spline_cpfa = make_interp_spline(x_data_fake2, y_cpfa_fake, k=SMOOTH_FACTOR)
    y_cpfa_smooth = spline_cpfa(x_smooth)
    cpfa_color = COLOR_1
    ax4 = ax1.twinx()
    ax4.set_ylabel(r'Embodied CFPA $(g\ CO_2e.q./mm^2)$', fontsize=AXIS_LABEL_FONT_SIZE, fontweight='bold', color=cpfa_color, labelpad=15)
    ax4.plot(x_smooth, y_cpfa_smooth, label='Embodied CFPA', linestyle = '--',color=cpfa_color, linewidth=LINE_WIDTH)
    ax4.plot(x_data, y_cpfa, 's', markersize=MARKER_SIZE, color=cpfa_color, alpha=0.8)
    ax4.spines['right'].set_color(cpfa_color)
    ax4.spines['right'].set_linewidth(2)
    ax4.spines['right'].set_position(('outward', 30))
    ax4.spines['left'].set_visible(False)
    ax4.spines['top'].set_visible(False)
    ax4.set_ylim(ax4.get_ylim()[0], ax4.get_ylim()[1] * 1.1)

    for label in ax4.get_yticklabels():
        label.set_color(cpfa_color)
        label.set_fontweight('bold')
        label.set_fontsize(TICK_FONT_SIZE)

    # Adjust the legend to include all series
    fig1.legend(loc='upper center', fontsize=AXIS_LABEL_FONT_SIZE, frameon=False, ncol=1, bbox_to_anchor=(0.3, 0.8))

    return fig1


In [ ]:
server_cpu = pd.read_csv('dataset/Intel-Datacenter-CPU.csv')
server_cpu['Emb CFP percentage'] = server_cpu['Emb CFP'] / server_cpu['Tot CFP']*100
server_cpu['Ope CFP percentage'] = server_cpu['Ope CFP'] / server_cpu['Tot CFP']*100
server_cpu['PassMark CPU score'] = server_cpu['PassMark CPU score'] / server_cpu['PassMark CPU score'].min()

server_cpu['Score / ECFPA'] = server_cpu['PassMark CPU score'] / server_cpu['ECFPA']
server_cpu['Score / ECFPA'] = server_cpu['Score / ECFPA'] / server_cpu['Score / ECFPA'].min()

server_cpu['Score / CFP'] = server_cpu['PassMark CPU score'] / server_cpu['Tot CFP']
server_cpu['Score / CFP'] = server_cpu['Score / CFP'] / server_cpu['Score / CFP'].min()

server_cpu.head()

In [14]:
chiplets = pd.read_csv("../dataset/chiplet_processors.csv")
chiplet_flagship_cpus = ["AMD Ryzen Threadripper 1950X", "AMD Ryzen Threadripper 3970X", "AMD Ryzen Threadripper 3990X", "AMD Ryzen Threadripper PRO 3995WX", "AMD Ryzen Threadripper PRO 5995WX"]
chiplet_flagship = chiplets[chiplets['Product'].isin(chiplet_flagship_cpus)]
chiplet_flagship

,Product,Type,Release Date,Process Size (nm),TDP (W),DieSizeValue,Die Size (mm^2),Transistors (million),Freq (GHz),Foundry,Vendor,#dies,Avg Die Area,Emb_Carbon_Mean,Ope_Carbon_Mean,Tot_Carbon_Mean
2,AMD Ryzen Threadripper 1950X,CPU,2017-08-10,14.0,180.0,426.0,2x 213,9600.0,3400.0,GlobalFoundries,AMD,2,213,8.052667,118.089995,126.142662
6,AMD Ryzen Threadripper 3970X,CPU,2019-11-25,7.0,280.0,148.0,4x 74,15200.0,3700.0,TSMC,AMD,4,74,3.842051,222.499136,226.341187
10,AMD Ryzen Threadripper 3990X,CPU,2020-02-07,7.0,280.0,592.0,8x 74,30400.0,2900.0,TSMC,AMD,8,74,31.086967,222.499136,253.586102
14,AMD Ryzen Threadripper PRO 3995WX,CPU,2020-07-14,7.0,280.0,592.0,8x 74,30400.0,2700.0,TSMC,AMD,8,74,31.086967,222.499136,253.586102
23,AMD Ryzen Threadripper PRO 5995WX,CPU,2022-03-08,7.0,280.0,648.0,8x 81,33200.0,2700.0,TSMC,AMD,8,81,37.801370,222.499136,260.300506


In [ ]:
chip_metrics_fig = chip_metrics_plot(server_cpu, server=True)
perf_metric_fig = performance_metric_plot(server_cpu, server=True)

perf_metric_fig.savefig('server_CPU_performance_metrics.pdf', bbox_inches='tight', dpi=300)
chip_metrics_fig.savefig('server_CPU_chip_metrics.pdf', bbox_inches='tight', dpi=300)

In [ ]:
desktop_cpu = pd.read_csv('dataset/Intel-Desktop-CPU.csv')

desktop_cpu['Emb CFP percentage'] = desktop_cpu['Emb CFP'] / desktop_cpu['Tot CFP']*100
desktop_cpu['Ope CFP percentage'] = desktop_cpu['Ope CFP'] / desktop_cpu['Tot CFP']*100


desktop_cpu['Single Core / ECFPA'] = desktop_cpu['Single Core Score'] / desktop_cpu['ECFPA']
desktop_cpu['Single Core / ECFPA'] = desktop_cpu['Single Core / ECFPA'] / desktop_cpu['Single Core / ECFPA'].min()


desktop_cpu['Single Core Score / CFP'] = desktop_cpu['Single Core Score'] / desktop_cpu['Tot CFP'].min()
desktop_cpu['Single Core Score / CFP'] = desktop_cpu['Single Core Score / CFP'] / desktop_cpu['Single Core Score / CFP'].min()


desktop_cpu['Multi-Core Score / ECFPA'] = desktop_cpu['Multi-Core Score'] / desktop_cpu['ECFPA']
desktop_cpu['Multi-Core Score / ECFPA'] = desktop_cpu['Multi-Core Score / ECFPA'] / desktop_cpu['Multi-Core Score / ECFPA'].min()

desktop_cpu['Multi-Core Score / CFP'] = desktop_cpu['Multi-Core Score'] / desktop_cpu['Tot CFP']
desktop_cpu['Multi-Core Score / CFP'] = desktop_cpu['Multi-Core Score / CFP'] / desktop_cpu['Multi-Core Score / CFP'].min()


desktop_cpu['Single Core Score'] = desktop_cpu['Single Core Score'] / desktop_cpu['Single Core Score'].min()
desktop_cpu['Multi-Core Score'] = desktop_cpu['Multi-Core Score'] / desktop_cpu['Multi-Core Score'].min()

desktop_cpu.head()

In [ ]:
chip_metrics_fig = chip_metrics_plot(desktop_cpu, server=False)
perf_metric_fig = performance_metric_plot(desktop_cpu, server=False)
perf_metric_fig.savefig('desktop_CPU_performance_metrics.pdf', bbox_inches='tight', dpi=300)
chip_metrics_fig.savefig('desktop_CPU_chip_metrics.pdf', bbox_inches='tight', dpi=300)